In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
ImageWidth=384
ImageHeight=384

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(60),
    transforms.Resize(size=(ImageWidth,ImageHeight)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # need to keep these transforms, at least do resize all images to same size and co
    ])